# Capstone Project

This capstone project allows me to implement a deep learning neural network to detect fingers from cam and count the number of extended fingers.
The main techniques used are; Threshold, Contour Detection, Convex Hull, Gaussian Blur.

In [1]:
from sklearn.metrics import pairwise
import cv2
import numpy as np

## Set global variables

In [ ]:
background = None
accumulated_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

## Calculate Accumulated average

It will update the running average of the background value in an ROI. It will detect objects when enters in.

First, grab ROI and get running average for 60 frames of video. After average value is calculated, we can put our hands in ROI.

In [ ]:
def calc_accum_avg(frame, accumulated_weight):
    global background
    
    # very first time
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

## Find segment

Grab hand segment from ROI

In [ ]:
def segment(frame, threshold_min = 25):
    diff = cv2.absdiff(background.astype('uint8'), frame)
    
    # apply threshold to img
    ret, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    # grab contour
    image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0: # didn't detect any contour
        return None
    else:
        # assume largest external contour in ROI is hand
        hand_segment = max(contours, key = cv2.contourArea)
        
        return (thresholded, hand_segment)

## Count fingers through Convex Hull

Convex Hull will draw a polygon around the hand. It connects points around most external points in a frame.

We need to get rid of wrist part. So we calculate the most extreme values (top, bottom, left, right).

Then use their intersection to get the center. Then calculate the distance the furthest away from the center. And use that distance to create a circle (in this case that distance will be the radius of that circle).  

Any points outside of the circle are extended fingers so we count them.

In [ ]:
def count_fingers(thresholded, hand_segments):
    conv_hull = cv2.convexHull(hand_segments)
    # top
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()[0]])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmin()[0]])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()[0]])
    right = tuple(conv_hull[conv_hull[:,:,0].argmin()[0]])
    
    # find center
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    distance = pairwise.euclidean_distances([cX, cY], Y = [left, right, top, bottom])[0]
    
    # get max distance
    max_distance = distance.max()
    
    # create a circle
    radius = int(0.9 * max_distance)
    circumference = (2 * np.pi * radius)
    
    # ROI for that circle (exclude color channel)
    circular_roi = np.zeros(thresholded[:2], dtype = 'uint8')
    
    # draw circle
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask = circular_roi)
    
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count = 0
    for cnt in contours:
        (x,y,w,h) = cv2.bondingRect(cnt)
        
        out_of_wrist = (cY + (cY * 0.25)) > (y + h)
        
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

## Run on live cam

In [ ]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    ret, frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray, (7,7), 0)
    
    if num_frames < 60: # for first 60 frames, 
        # calculate accumulated 
        calc_accum_avg(gray, accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy, 'WAIT. Getting Background....', (200,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),3)
            cv2.imshow('Finger Counts', frame_copy)
    else:
        hand = segment(gray)
        
        if hand is not None:
            thresholded, hand_segment = hand
            
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255,0,0), 5)
            
            fingers = count_fingers(thresholded, hand_segment)
            
            cv2.putText(frame_copy, str(fingers), (70,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,255),2)
            
            cv2.imshow('Thresholded', thresholded)
            
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,200), 5)
    
    num_frames += 1
    
    cv2.imshow('Finger Counts', frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cam.release()
            